# 12/4/19

Creating a MIMIC CSV for drain detection

In [1]:
import os
import os.path as osp

import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
mimic_with_tubes_df = pd.read_csv(all_mimic_data_with_tubes_v2.tsv', sep='\t', index_col=0)
mimic_df = pd.read_csv('/lfs/1/jdunnmon/repos/mi-subsets/mimic/all_mimic_data.tsv', sep='\t', index_col=0)


In [6]:
mimic_with_tubes_df

,subject_id,study_id,path_report,raw_report,dicom_id,path_image,split,PerformedProcedureStepDescription,ViewPosition,Rows,...,Enlarged Cardiomediastinum Negbio,Fracture Negbio,Lung Lesion Negbio,Lung Opacity Negbio,No Finding Negbio,Pleural Effusion Negbio,Pleural Other Negbio,Pneumonia Negbio,Pneumothorax Negbio,Support Devices Negbio
242,10003400,50100991,files/p10/p10003400/s50100991.txt,Resolved right pleural effusion with small-mod...,c1981851-eee51654-80b71d01-2ee65952-4c70c720,files/p10/p10003400/s50100991/c1981851-eee5165...,train,CHEST (PORTABLE AP),AP,2544,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0
243,10003400,50100991,files/p10/p10003400/s50100991.txt,Resolved right pleural effusion with small-mod...,ed7dc3ef-9ed80b39-d9ae643a-964c37dc-87f73a50,files/p10/p10003400/s50100991/ed7dc3ef-9ed80b3...,train,CHEST (PORTABLE AP),AP,2544,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0
246,10003400,51127691,files/p10/p10003400/s51127691.txt,EXAMINATION: CHEST (PORTABLE AP) INDICATION: ...,e730732b-c35b3296-bd65ed0a-92becc50-25be5475,files/p10/p10003400/s51127691/e730732b-c35b329...,train,CHEST (PORTABLE AP),AP,2539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
257,10003400,53341253,files/p10/p10003400/s53341253.txt,EXAMINATION: CHEST (PORTABLE AP) INDICATION: ...,19b74f79-7ad317b1-887496c3-2645db29-48245a51,files/p10/p10003400/s53341253/19b74f79-7ad317b...,train,CHEST (PORTABLE AP),AP,2539,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,1.0,1.0
265,10003400,56185120,files/p10/p10003400/s56185120.txt,EXAMINATION: CHEST (PORTABLE AP) INDICATION: ...,7eea27b8-a4127ebb-b814b969-61d6fe59-2a1bd17e,files/p10/p10003400/s56185120/7eea27b8-a4127eb...,train,CHEST (PORTABLE AP),AP,2791,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376973,19997367,55253335,files/p19/p19997367/s55253335.txt,CHEST RADIOGRAPH INDICATION: Status post righ...,9560e593-004e0e78-e4fa3581-09e340db-5bcfae99,files/p19/p19997367/s55253335/9560e593-004e0e7...,train,CHEST (PORTABLE AP),AP,3056,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,1.0
376977,19997367,55414125,files/p19/p19997367/s55414125.txt,INDICATION: ___ year old woman with second ch...,7ef70053-2152bf8e-fa9a7f42-cb2ad4c0-4c8bcf97,files/p19/p19997367/s55414125/7ef70053-2152bf8...,train,CHEST (PORTABLE AP),AP,3050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
376990,19997367,56627054,files/p19/p19997367/s56627054.txt,Compared to the prior radiograph performed 7 h...,1c406bcd-33ae2acd-d72d0cb5-891d4a1b-b0ca8fbd,files/p19/p19997367/s56627054/1c406bcd-33ae2ac...,train,CHEST (PORTABLE AP),AP,3050,...,NaN,NaN,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,1.0
376991,19997367,56627054,files/p19/p19997367/s56627054.txt,Compared to the prior radiograph performed 7 h...,61b54780-b5b3aeac-2e65ca06-fcd8654e-40dd935d,files/p19/p19997367/s56627054/61b54780-b5b3aea...,train,CHEST (PORTABLE AP),AP,3050,...,NaN,NaN,NaN,1.0,NaN,-1.0,NaN,NaN,NaN,1.0
